In [1]:
import pandas as pd
import numpy as np
import pickle as p

In [2]:
data_all = pd.read_csv('../data_all.csv')

# Test with Subset of Data
np.random.seed(43)
item_subset = list(np.random.choice(data_all['Item_ID'].unique(), size=10, replace=False))
data_subset = data_all.loc[data_all['Item_ID'].isin(item_subset)].copy()

# # Test with All Data
# data_subset = data_all.copy()

In [3]:
# item_subset

### `/train`

In [4]:
import time

In [5]:
data_subset

,Wk,Tier,Groups,Store,Item_ID,Qty_,Price_
0,26,25,53,1973,16019,37.0,14.15
1,25,25,53,1973,16019,60.0,14.15
2,24,25,53,1973,16019,47.0,14.15
3,23,25,53,1973,16019,68.0,14.10
4,22,25,53,1973,16019,115.0,14.05
...,...,...,...,...,...,...,...
3867650,5,22,1,2,202,1856.0,2.10
3867651,4,22,1,2,202,1883.0,2.10
3867652,3,22,1,2,202,1773.0,2.10
3867653,2,22,1,2,202,1990.0,2.10


In [6]:
import requests
import json
from threading import Thread

url = 'https://fillet.azurewebsites.net/train/'
# url = 'http://localhost:5000/train/'

data_subset.to_parquet('data_subset.parquet')
files = {'data': open('data_subset.parquet', 'rb')}

payload = {'cv_acc': True, 'project_id': 'aaaa'}

Thread(target=requests.post,
       args=(url, ),
       kwargs={
           'data': payload,
           'files': files
       }).start()

time.sleep(10)
files['data'].close()

### `/query_progress`

In [12]:
import requests
import json

url = 'https://fillet.azurewebsites.net/query_progress/'
# url = 'http://localhost:5000/query_progress/'


payload = {'project_id': 'aaaa'}

headers = {'content-type': 'application/json',
           'Accept-Charset': 'UTF-8'
          }

payload = json.dumps(payload)
r = requests.post(url, data=payload,headers=headers,  timeout=3)

print('% Complete:',round(r.json()['pct_complete'],3))
print('CV % Complete:',round(r.json()['cv_progress'],3))
print('CV Done:',r.json()['cv_done'])

ReadTimeout: HTTPSConnectionPool(host='fillet.azurewebsites.net', port=443): Read timed out. (read timeout=3)

In [110]:
set(r.json()['project_items']) - set(r.json()['train_complete'])

set()

### `/get_cv_results`

In [14]:
import requests
import json

url = 'https://fillet.azurewebsites.net/get_cv_results/'
# url = 'http://localhost:5000/get_cv_results/'


payload = {'project_id': 'aaaa'}

headers = {'content-type': 'application/json',
           'Accept-Charset': 'UTF-8'
          }

payload = json.dumps(payload)
r = requests.post(url, data=payload,headers=headers,  timeout=5)
pd.DataFrame().from_dict((json.loads(r.json())))

,item_id,avg_sales,r2_score,mae_score,mpe_score,rmse_score
0,1195.0,129.492813,0.633854,29.109335,0.224795,38.164598
1,1230.0,8.996674,0.522868,4.356677,0.484254,5.787544
2,16019.0,109.613182,-0.004587,32.791393,0.299156,41.253508
3,202.0,2110.440186,0.734161,330.306183,0.156511,457.301697
4,2531.0,318.983582,0.517900,82.751486,0.259422,110.458474
5,2589.0,161.616043,0.755766,30.343028,0.187748,41.519588
6,3732.0,126.142509,0.677597,22.660966,0.179646,29.741242
7,3743.0,132.480972,0.695241,30.203218,0.227982,40.135818
8,3766.0,91.448227,0.682981,20.327661,0.222286,28.310535


In [14]:
json.loads(r.json())

{'item_id': {'0': 3722.0,
  '1': 3723.0,
  '2': 7110.0,
  '3': 2313.0,
  '4': 201.0,
  '5': 350.0,
  '6': 1204.0,
  '7': 3726.0,
  '8': 204.0,
  '9': 3738.0,
  '10': 1221.0,
  '11': 1502.0,
  '12': 2258.0,
  '13': 1503.0,
  '14': 7109.0,
  '15': 1231.0,
  '16': 16019.0,
  '17': 2256.0,
  '18': 1203.0,
  '19': 2257.0,
  '20': 3736.0,
  '21': 16017.0,
  '22': 16018.0,
  '23': 3743.0,
  '24': 1205.0,
  '25': 3727.0,
  '26': 3737.0,
  '27': 203.0,
  '28': 7078.0,
  '29': 1230.0,
  '30': 196.0,
  '31': 1615.0,
  '32': 3773.0,
  '33': 1225.0,
  '34': 202.0,
  '35': 7108.0,
  '36': 200.0,
  '37': 7079.0,
  '38': 3742.0,
  '39': 3821.0,
  '40': 1002.0,
  '41': 3728.0,
  '42': 3771.0,
  '43': 1232.0,
  '44': 1003.0,
  '45': 3767.0,
  '46': 197.0,
  '47': 3732.0,
  '48': 1153.0,
  '49': 7080.0,
  '50': 3817.0,
  '51': 1195.0,
  '52': 281.0,
  '53': 1504.0,
  '54': 3731.0,
  '55': 7113.0,
  '56': 284.0,
  '57': 3733.0,
  '58': 3766.0,
  '59': 3772.0,
  '60': 1152.0,
  '61': 3816.0,
  '62': 2315.0

In [127]:
r.json()

{'status': 'incomplete'}

### `/optimize`

In [ ]:
url = 'http://localhost:5000/optimize/'

payload = {'project_id': 1,
           'constraints': [],
           'population': 100,
           'max_epoch': 200
           }

headers = {'content-type': 'application/json',
           'Accept-Charset': 'UTF-8'
          }

payload = json.dumps(payload)
r = requests.post(url, data=payload, headers=headers)

In [ ]:
best_prices = r.json()['result']
best_prices

In [ ]:
import requests
import json

url = 'http://localhost:5000/predict/'
payload = {'prices': best_prices,
           'project_id': 1
           }
headers = {'content-type': 'application/json',
           'Accept-Charset': 'UTF-8'
          }
payload = json.dumps(payload)
r = requests.post(url, data=payload, headers=headers)

# What If Prediction
qty = r.json()['qty_estimates']

In [ ]:
qty

In [ ]:
# Calculate Revenue
revenue = 0
for item in best_prices.keys():
    revenue += (best_prices[item]*qty['Qty_'+item])
    
print(revenue)

### `/predict/`

In [16]:
# Sample Actual Price 
from helper_functions import *

sales_data = data_subset

sales_data = optimize_memory(sales_data)

sales_data_wide = sales_data.set_index(
    ['Wk', 'Tier', 'Store', 'Item_ID']).unstack(level=-1).reset_index().copy()
sales_data_wide.columns = [
    ''.join(str(i) for i in col).strip()
    for col in sales_data_wide.columns.values
]
sales_data_wide = sales_data_wide.sort_values(
    ['Tier', 'Store', 'Wk'], ascending=True).reset_index(drop=True)

sales_data_wide_clean = sales_data_wide.dropna(axis=0).copy()
dataset = sales_data_wide_clean
price_columns = [
    col for col in sales_data_wide_clean.columns if col.startswith('Price')
]

a = dataset.sample(1)
a_dict = a[price_columns].iloc[0].to_dict()
a_input = {}
for key in a_dict.keys():
    a_input[key.split('_')[1]]=round(a_dict[key],2)

print('Input Prices Quantities:')
a_input




Input Prices Quantities:


{'202': 2.35,
 '1195': 3.15,
 '1230': 3.25,
 '2531': 6.25,
 '2589': 11.9,
 '3726': 9.95,
 '3732': 11.55,
 '3743': 11.1,
 '3766': 9.55,
 '16019': 13.6}

In [19]:
# Predict Sales Qty and Compare to Actual

import requests
import json

url = 'https://fillet.azurewebsites.net/predict/'
# url = 'http://localhost:5000/predict/'

payload = {'prices': a_input, 'project_id': 7}

headers = {'content-type': 'application/json', 'Accept-Charset': 'UTF-8'}

payload = json.dumps(payload)
r = requests.post(url, data=payload, headers=headers)
qty = r.json()['qty_estimates']
qty_outp = {}
for key in qty.keys():
    qty_outp[key] = round(qty[key], 0)
actual = a[[
    col for col in sales_data_wide_clean.columns if col.startswith('Qty_')
]].iloc[0].to_dict()
pd.DataFrame([qty_outp, actual], index=['Estimated', 'Actual']).transpose()

,Estimated,Actual
Qty_1195,223.0,195.0
Qty_1230,19.0,17.0
Qty_16019,210.0,210.0
Qty_202,5193.0,4785.0
Qty_2531,517.0,703.0
Qty_2589,419.0,277.0
Qty_3726,243.0,254.0
Qty_3732,342.0,350.0
Qty_3743,210.0,242.0
Qty_3766,241.0,227.0


In [32]:
r.json()

{'qty_estimates': {'Qty_1230': 17,
  'Qty_16019': 205,
  'Qty_202': 5244,
  'Qty_3732': 344,
  'Qty_3766': 236}}